In [ ]:
!pip install pykan

In [1]:
#@title 1.3. IMPORT LIBRARY
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

In [2]:
import sys
sys.path.append('../utils')
from data_retrieval import credit_data_retrieval
from neural_net import *

In [3]:
#@title 1.4. RETRIEVE DATA
df = credit_data_retrieval()
df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,Risk
0,0.146949,-0.745131,-1.236478,0.176948,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1,0.146949,0.949817,2.248194,-0.284901,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
2,-1.383771,-0.416562,-0.738668,0.045495,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
3,0.146949,1.634247,1.750384,0.130233,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
4,0.146949,0.566664,0.256953,0.229637,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
from kan import *
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
# model = KAN(width=[24, 1], grid=3, k=3, seed=42, device=device)

cpu


In [6]:
from sklearn.model_selection import StratifiedKFold
import torch.optim as optim
import torch.nn as nn

learning_rates = [0.1]
accuracy_train = []
accuracy_test = []

# Set up 10-fold stratified cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(skf.split(df, df['Risk'])):
    print(f"Fold {fold + 1}")

    # Split the data into training and testing sets based on the current fold
    df_train = df.iloc[train_index]
    df_test = df.iloc[test_index]

    # Create datasets from the train and test sets
    dataset_train = CreditDataSet(df_train)
    dataset_test = CreditDataSet(df_test)

    # Create DataLoaders for training and testing
    dataloader_train = DataLoader(
        dataset_train,
        batch_size=16,
        shuffle=True,
    )
    dataloader_test = DataLoader(
        dataset_test,
        batch_size=16,
        shuffle=False,
    )

    # Iterate over the learning rates (although only one in this example)
    for lr in learning_rates:
        net = Net(29)  # Re-initialize the network for each fold and learning rate
        optimizer = optim.SGD(net.parameters(), lr=lr)
        criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss

        # Train the model on the current training fold
        train_model(
            optimizer=optimizer,
            net=net,
            criterion=criterion,
            num_epochs=500,  # Adjust as needed
            dataloader_train=dataloader_train,
        )

        # Evaluate the model on both the training and test sets
        accuracy = evaluate_accuracy(net, dataloader_train)
        val = evaluate_accuracy(net, dataloader_test)

        print(f"Learning Rate: {lr}, Train Accuracy: {accuracy}, Test Accuracy: {val}")

        # Store the accuracy results
        accuracy_train.append(accuracy)
        accuracy_test.append(val)

# Optionally, summarize results across all folds
print(f"Average Train Accuracy: {sum(accuracy_train) / len(accuracy_train):.3f}")
print(f"Average Test Accuracy: {sum(accuracy_test) / len(accuracy_test):.3f}")


Fold 1


100%|██████████| 500/500 [00:16<00:00, 30.46it/s]


Learning Rate: 0.1, Train Accuracy: 0.8787500262260437, Test Accuracy: 0.7350000143051147
Fold 2


100%|██████████| 500/500 [00:16<00:00, 30.45it/s]


Learning Rate: 0.1, Train Accuracy: 0.8762500286102295, Test Accuracy: 0.7149999737739563
Fold 3


...

Learning Rate: 0.1, Train Accuracy: 0.8725000023841858, Test Accuracy: 0.7200000286102295
Fold 4


100%|██████████| 500/500 [00:16<00:00, 30.55it/s]


Learning Rate: 0.1, Train Accuracy: 0.8387500047683716, Test Accuracy: 0.7400000095367432
Fold 5


100%|██████████| 500/500 [00:16<00:00, 30.64it/s]


Learning Rate: 0.1, Train Accuracy: 0.862500011920929, Test Accuracy: 0.7099999785423279
Average Train Accuracy: 0.866
Average Test Accuracy: 0.724


In [ ]:
learning_rates = [0.1]
accuracy_train = []
accuracy_test = []

# Perform grid search over learning rates
for lr in learning_rates:
    net = Net()  # Re-initialize the network for each experiment
    optimizer = optim.SGD(net.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()  # Use standard binary cross-entropy

    train_model(
        optimizer=optimizer,
        net=net,
        criterion=criterion,
        num_epochs=200,  # Use fewer epochs for quick testing
        dataloader_train=dataloader_train
    )
    
    # Evaluate accuracy on training data (or use a test/validation set if available)
    accuracy = evaluate_accuracy(net, dataloader_train)
    val = evaluate_accuracy(net, dataloader_test)

    print(f"Learning Rate: {lr}, Accuracy: {accuracy}, Test: {val}")

    accuracy_train.append(accuracy)
    accuracy_test.append(val)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(learning_rates, accuracy_train, marker='o', linestyle='--', color='b', label='Accuracy')
plt.plot(learning_rates, accuracy_test, marker='o', linestyle='--', color='r', label='Accuracy')

plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Learning Rate')
plt.legend()
plt.grid(True)
plt.show()